In [5]:
import os

import torch
from pytorch_lightning import seed_everything

In [6]:
from bci_aic3.models.eegnet import EEGNet
from bci_aic3.paths import (
    MI_CONFIG_PATH,
    MI_RUNS_DIR,
    RAW_DATA_DIR,
    SSVEP_CONFIG_PATH,
    SSVEP_RUNS_DIR
)
from bci_aic3.train import train_model
from bci_aic3.util import save_model, load_model

In [7]:

# Code necessary to create reproducible runs
os.environ["CUBLAS_WORKSPACE_CONFIG"] = ":4096:8"
seed_everything(42, workers=True)
torch.use_deterministic_algorithms(True, warn_only=True)


Seed set to 42


In [8]:
trainer, model = train_model(model=EEGNet, config_path=MI_CONFIG_PATH)

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
p:\Programming\AIC3\repo\bci_aic3\.venv\Lib\site-packages\pytorch_lightning\callbacks\model_checkpoint.py:654: Checkpoint directory P:\Programming\AIC3\repo\bci_aic3\checkpoints\MI exists and is not empty.

  | Name           | Type               | Params | Mode 
--------------------------------------------------------------
0 | model          | EEGNet             | 36.2 K | train
1 | criterion      | CrossEntropyLoss   | 0      | train
2 | train_accuracy | MulticlassAccuracy | 0      | train
3 | val_accuracy   | MulticlassAccuracy | 0      | train
4 | train_f1       | MulticlassF1Score  | 0      | train
5 | val_f1         | MulticlassF1Score  | 0      | train
--------------------------------------------------------------
36.2 K    Trainable params
0         Non-trainable params
36.2 K    Total params
0.145     Total estimated model params size (MB)
20        Modules in train 

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Metric val_loss improved. New best score: 0.862
Epoch 0, global step 75: 'val_f1' reached 0.43182 (best 0.43182), saving model to 'P:\\Programming\\AIC3\\repo\\bci_aic3\\checkpoints\\MI\\eegnet-mi-best-f1-val_f1=0.4318-epoch=00-v3.ckpt' as top 3
`Trainer.fit` stopped: `max_epochs=1` reached.


In [ ]:
val_f1_score = trainer.callback_metrics.get("val_f1")

if val_f1_score is not None:
    val_f1_score = val_f1_score.

0.4318181872367859

In [5]:
save_model(model, SSVEP_RUNS_DIR / "model_scripted.pt")

In [6]:
loaded_model = load_model(SSVEP_RUNS_DIR / "model_scripted.pt")  # Save

In [7]:
from bci_aic3.inference import (
    create_inference_data_loader,
    load_and_preprocess_for_inference,
    predict_batch,
)


In [8]:
test_dataset = load_and_preprocess_for_inference(
    "test.csv", base_path=RAW_DATA_DIR, task_type="SSVEP"
)

test_loader = create_inference_data_loader(test_dataset)

100%|██████████| 50/50 [00:02<00:00, 22.94it/s]


In [9]:
preds = predict_batch(model=loaded_model,
                      data_loader=test_loader,
                      device="cpu")
preds

[2,
 3,
 1,
 1,
 2,
 1,
 1,
 1,
 1,
 2,
 0,
 0,
 0,
 0,
 1,
 1,
 2,
 2,
 2,
 2,
 3,
 3,
 1,
 0,
 0,
 3,
 3,
 3,
 3,
 3,
 0,
 3,
 3,
 3,
 0,
 3,
 1,
 3,
 1,
 3,
 0,
 3,
 3,
 3,
 3,
 3,
 3,
 2,
 2,
 3]

In [24]:
from bci_aic3.inference import make_inference

ssvep_preds = make_inference(model=loaded_model,
                       csv_file="test.csv",
                       base_path=RAW_DATA_DIR,
                       task_type="SSVEP",
                       reverse_mapping=True)
ssvep_preds

100%|██████████| 50/50 [00:02<00:00, 23.63it/s]


['Forward',
 'Backward',
 'Right',
 'Right',
 'Forward',
 'Right',
 'Right',
 'Right',
 'Right',
 'Forward',
 'Left',
 'Left',
 'Left',
 'Left',
 'Right',
 'Right',
 'Forward',
 'Forward',
 'Forward',
 'Forward',
 'Backward',
 'Backward',
 'Right',
 'Left',
 'Left',
 'Backward',
 'Backward',
 'Backward',
 'Backward',
 'Backward',
 'Left',
 'Backward',
 'Backward',
 'Backward',
 'Left',
 'Backward',
 'Right',
 'Backward',
 'Right',
 'Backward',
 'Left',
 'Backward',
 'Backward',
 'Backward',
 'Backward',
 'Backward',
 'Backward',
 'Forward',
 'Forward',
 'Backward']

In [31]:
loaded_mi_model = load_model(MI_RUNS_DIR / "model_scripted.pt")

mi_preds = make_inference(model=loaded_mi_model,
                       csv_file="test.csv",
                       base_path=RAW_DATA_DIR,
                       task_type="MI",
                       reverse_mapping=True)
mi_preds

100%|██████████| 50/50 [00:02<00:00, 17.60it/s]


['Left',
 'Left',
 'Right',
 'Left',
 'Right',
 'Left',
 'Right',
 'Right',
 'Left',
 'Left',
 'Right',
 'Right',
 'Left',
 'Left',
 'Left',
 'Left',
 'Right',
 'Right',
 'Right',
 'Right',
 'Right',
 'Left',
 'Left',
 'Left',
 'Right',
 'Right',
 'Right',
 'Left',
 'Left',
 'Left',
 'Right',
 'Left',
 'Left',
 'Left',
 'Right',
 'Right',
 'Left',
 'Left',
 'Right',
 'Left',
 'Left',
 'Left',
 'Right',
 'Left',
 'Left',
 'Right',
 'Left',
 'Left',
 'Right',
 'Right']

In [ ]:
import pandas as pd


df = pd.read_csv(RAW_DATA_DIR / "test.csv")

In [ ]:

df.loc[df["task"] == "SSVEP", "labels"] = ssvep_preds
df.loc[df["task"] == "MI", "labels"] = mi_preds
df

In [38]:
from bci_aic3.paths import SUBMISSIONS_DIR


df[['id', 'labels']].to_csv(SUBMISSIONS_DIR / 'quick_test_1_epoch.csv', index=False)